In [3]:
import pandas as pd

In [4]:
from sklearn.ensemble import RandomForestRegressor 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import  mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
import numpy as np

C:\Users\Arpita\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
C:\Users\Arpita\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
cdsSpread_data = pd.read_stata("C:\\Users\\Arpita\\Desktop\\Book\\Fall 2018\\Machine Learning\\Assignment\\cds_spread5y_2001_2016.dta")


In [6]:
cdsSpread_data.shape

(169043, 9)

In [7]:
cdsSpread_data.dtypes

gvkey                    object
mdate            datetime64[ns]
redcode                  object
tier                     object
ticker                   object
shortname                object
docclause                object
spread5y                float64
impliedrating            object
dtype: object

In [8]:
cdsSpread_data.head()

,gvkey,mdate,redcode,tier,ticker,shortname,docclause,spread5y,impliedrating
0,001038,2004-08-31,0A17CF,SNRFOR,AEN,AMC Entmt Inc,MR,0.042000,BB
1,001038,2005-08-31,0A17CF,SNRFOR,AEN,AMC Entmt Inc,XR,0.034000,BB
2,001038,2005-09-30,0A17CF,SNRFOR,AEN,AMC Entmt Inc,XR,0.034000,BB
3,001038,2005-10-31,0A17CF,SNRFOR,AEN,AMC Entmt Inc,XR,0.034001,BB
4,001038,2005-11-30,0A17CF,SNRFOR,AEN,AMC Entmt Inc,XR,0.034001,BB


In [9]:
#Seperate date based on month & years and create quarters accordingly 
cdsSpread_data['Date'] = pd.to_datetime(cdsSpread_data['mdate'])
cdsSpread_data['Month']= cdsSpread_data['Date'].dt.month
cdsSpread_data['Year']=cdsSpread_data['Date'].dt.year

cdsSpread_data['quarter']='4'


In [10]:
cdsSpread_data.loc[cdsSpread_data['Month']>9,"quarter"]=4
cdsSpread_data.loc[(cdsSpread_data['Month']>6) & (cdsSpread_data['Month']<=9),"quarter"]=3
cdsSpread_data.loc[(cdsSpread_data['Month']>3) & (cdsSpread_data['Month']<=6),"quarter"]=2
cdsSpread_data.loc[(cdsSpread_data['Month'])<=3,"quarter"]=1

In [11]:
cdsSpread_data['gvkey'] = cdsSpread_data['gvkey'].astype(float)
cdsSpread_data['quarter'] = cdsSpread_data['quarter'].astype(float)
cdsSpread_data['Year'] = cdsSpread_data['Year'].astype(float)

In [12]:
qtr_Mdata = pd.read_csv("C:\\Users\\Arpita\\Downloads\\Quarterly Merged CRSP-Compustat.csv", low_memory = False)

In [13]:
qtr_Mdata.shape

(58928, 687)

In [14]:
qtr_Mdata.dtypes

GVKEY          int64
LINKPRIM      object
LIID          object
LINKTYPE      object
LPERMNO        int64
LPERMCO        int64
LINKDT        object
LINKENDDT     object
datadate      object
fyearq         int64
fqtr           int64
fyr            int64
indfmt        object
consol        object
popsrc        object
datafmt       object
tic           object
cusip         object
conm          object
acctchgq      object
acctstdq      object
adrrq        float64
ajexq        float64
ajpq         float64
bsprq         object
compstq       object
curcdq        object
curncdq       object
currtrq      float64
curuscnq     float64
              ...   
addzip        object
busdesc       object
city          object
conml         object
county        object
dlrsn        float64
ein           object
fax           object
fyrc           int64
ggroup         int64
gind           int64
gsector        int64
gsubind        int64
idbflag       object
incorp        object
loc           object
naics        

In [15]:
qtr_Mdata.head()

,GVKEY,LINKPRIM,LIID,LINKTYPE,LPERMNO,LPERMCO,LINKDT,LINKENDDT,datadate,fyearq,...,priusa,sic,spcindcd,spcseccd,spcsrc,state,stko,weburl,dldte,ipodate
0,1038,P,01,LU,66413,6301,08/18/1983,12/31/2004,03/31/2001,2000,...,1,7830,245.0,978.0,NaN,MO,0.0,www.amctheatres.com,12/27/2004,NaN
1,1038,P,01,LU,66413,6301,08/18/1983,12/31/2004,06/30/2001,2001,...,1,7830,245.0,978.0,NaN,MO,0.0,www.amctheatres.com,12/27/2004,NaN
2,1038,P,01,LU,66413,6301,08/18/1983,12/31/2004,09/30/2001,2001,...,1,7830,245.0,978.0,NaN,MO,0.0,www.amctheatres.com,12/27/2004,NaN
3,1038,P,01,LU,66413,6301,08/18/1983,12/31/2004,12/31/2001,2001,...,1,7830,245.0,978.0,NaN,MO,0.0,www.amctheatres.com,12/27/2004,NaN
4,1038,P,01,LU,66413,6301,08/18/1983,12/31/2004,03/31/2002,2001,...,1,7830,245.0,978.0,NaN,MO,0.0,www.amctheatres.com,12/27/2004,NaN


In [16]:
qtr_Mdata=qtr_Mdata.rename(columns = {'GVKEY':'gvkey'})
qtr_Mdata=qtr_Mdata.rename(columns = {'datadate':'mdate'})

In [17]:
qtr_Mdata['mdate'].unique()

array(['03/31/2001', '06/30/2001', '09/30/2001', '12/31/2001',
       '03/31/2002', '06/30/2002', '09/30/2002', '12/31/2002',
       '03/31/2003', '06/30/2003', '09/30/2003', '12/31/2003',
       '03/31/2004', '06/30/2004', '09/30/2004', '12/31/2004',
       '03/31/2005', '06/30/2005', '09/30/2005', '12/31/2005',
       '03/31/2006', '06/30/2006', '09/30/2006', '12/31/2006',
       '03/31/2007', '06/30/2007', '09/30/2007', '12/31/2007',
       '03/31/2008', '06/30/2008', '09/30/2008', '12/31/2008',
       '03/31/2009', '06/30/2009', '09/30/2009', '12/31/2009',
       '03/31/2010', '06/30/2010', '09/30/2010', '12/31/2010',
       '03/31/2011', '06/30/2011', '09/30/2011', '12/31/2011',
       '12/31/2013', '03/31/2014', '06/30/2014', '09/30/2014',
       '12/31/2014', '03/31/2015', '06/30/2015', '09/30/2015',
       '12/31/2015', '03/31/2016', '06/30/2016', '09/30/2016',
       '12/31/2016', '03/31/2017', '03/31/2012', '06/30/2012',
       '09/30/2012', '12/31/2012', '03/31/2013', '06/30

In [19]:
#Seperate date based on month & years and create quarters accordingly 
qtr_Mdata['Date'] = pd.to_datetime(qtr_Mdata['mdate'])
qtr_Mdata['Month']= qtr_Mdata['Date'].dt.month
qtr_Mdata['Year']=qtr_Mdata['Date'].dt.year

qtr_Mdata['quarter']='4'


In [20]:
qtr_Mdata.loc[qtr_Mdata['Month']>9,"quarter"]=4
qtr_Mdata.loc[(qtr_Mdata['Month']>6) & (qtr_Mdata['Month']<=9),"quarter"]=3
qtr_Mdata.loc[(qtr_Mdata['Month']>3) & (qtr_Mdata['Month']<=6),"quarter"]=2
qtr_Mdata.loc[(qtr_Mdata['Month'])<=3,"quarter"]=1

In [21]:
qtr_Mdata['gvkey'] = qtr_Mdata['gvkey'].astype(float)
qtr_Mdata['quarter'] = qtr_Mdata['quarter'].astype(float)
qtr_Mdata['Year'] = qtr_Mdata['Year'].astype(float)

In [22]:
qtr_Mdata['quarter'].head()

0    1.0
1    2.0
2    3.0
3    4.0
4    1.0
Name: quarter, dtype: float64

In [23]:
data = pd.merge(cdsSpread_data, qtr_Mdata, on=['gvkey', 'quarter', 'Year'])

In [30]:
#Use only numerical records 
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
data = data.select_dtypes(include=numerics)

#Pre processing data: replace NA with median 
data=data.fillna(data.median())
data=data.dropna(axis=1, how='any')

X=data.drop('spread5y', axis=1)

y=data['spread5y']
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train


,gvkey,Month_x,Year,quarter,LPERMNO,LPERMCO,fyearq,fqtr,fyr,adrrq,...,gind,gsector,gsubind,naics,priusa,sic,spcindcd,spcseccd,stko,Month_y
70943,14538.0,11,2016.0,4.0,75652,5310,2016,4,12,1.0,...,352020,35,35202010,325412,90,2834,285.0,905.0,0.0,12
30215,5894.0,6,2002.0,2.0,33072,20970,2002,2,12,4.0,...,151010,15,15101050,325,90,2800,160.0,970.0,3.0,6
93939,61739.0,6,2007.0,2.0,82775,31386,2007,2,12,1.0,...,403010,40,40301030,524126,1,6331,830.0,800.0,0.0,6
110194,174490.0,10,2011.0,4.0,91391,50964,2011,4,12,1.0,...,501010,50,50101020,517110,1,4813,382.0,935.0,0.0,12
109958,165675.0,8,2013.0,3.0,91063,50032,2013,4,9,1.0,...,254010,25,25401030,515120,1,4833,382.0,935.0,0.0,9
71514,14790.0,11,2016.0,4.0,75342,20642,2016,4,12,1.0,...,252010,25,25201030,236117,1,1531,305.0,976.0,0.0,12
77985,19565.0,12,2005.0,4.0,76221,26084,2005,4,12,4.0,...,151040,15,15104020,212,90,1000,360.0,970.0,0.0,12
109055,160329.0,5,2008.0,2.0,90319,45483,2008,2,12,1.0,...,451010,45,45101010,519130,1,7370,382.0,935.0,0.0,6
13992,3105.0,12,2007.0,4.0,24046,6996,2007,4,12,1.0,...,254010,25,25401020,515112,2,4832,150.0,978.0,3.0,12
104779,140760.0,7,2005.0,3.0,88668,40148,2005,4,8,1.0,...,151010,15,15101030,111,1,100,167.0,970.0,0.0,8


In [31]:
X_test

,gvkey,Month_x,Year,quarter,LPERMNO,LPERMCO,fyearq,fqtr,fyr,adrrq,...,gind,gsector,gsubind,naics,priusa,sic,spcindcd,spcseccd,stko,Month_y
70943,14538.0,11,2016.0,4.0,75652,5310,2016,4,12,1.0,...,352020,35,35202010,325412,90,2834,285.0,905.0,0.0,12
30215,5894.0,6,2002.0,2.0,33072,20970,2002,2,12,4.0,...,151010,15,15101050,325,90,2800,160.0,970.0,3.0,6
93939,61739.0,6,2007.0,2.0,82775,31386,2007,2,12,1.0,...,403010,40,40301030,524126,1,6331,830.0,800.0,0.0,6
110194,174490.0,10,2011.0,4.0,91391,50964,2011,4,12,1.0,...,501010,50,50101020,517110,1,4813,382.0,935.0,0.0,12
109958,165675.0,8,2013.0,3.0,91063,50032,2013,4,9,1.0,...,254010,25,25401030,515120,1,4833,382.0,935.0,0.0,9
71514,14790.0,11,2016.0,4.0,75342,20642,2016,4,12,1.0,...,252010,25,25201030,236117,1,1531,305.0,976.0,0.0,12
77985,19565.0,12,2005.0,4.0,76221,26084,2005,4,12,4.0,...,151040,15,15104020,212,90,1000,360.0,970.0,0.0,12
109055,160329.0,5,2008.0,2.0,90319,45483,2008,2,12,1.0,...,451010,45,45101010,519130,1,7370,382.0,935.0,0.0,6
13992,3105.0,12,2007.0,4.0,24046,6996,2007,4,12,1.0,...,254010,25,25401020,515112,2,4832,150.0,978.0,3.0,12
104779,140760.0,7,2005.0,3.0,88668,40148,2005,4,8,1.0,...,151010,15,15101030,111,1,100,167.0,970.0,0.0,8


In [32]:
X_test= X_test.drop('Month_x', axis=1)
X_test= X_test.drop('Month_y', axis=1)
X_test= X_test.drop('quarter', axis=1)
X_test= X_test.drop('Year', axis=1)
X_test= X_test.drop('gvkey', axis=1)

TrainData=data[(data['Year'] < 2010)]

#splitting x and y for test data
X_train= TrainData.drop('spread5y', axis=1)
y_train=TrainData['spread5y']
X_train= X_train.drop('Month_x', axis=1)
X_train= X_train.drop('Month_y', axis=1)
X_train= X_train.drop('quarter', axis=1)
X_train= X_train.drop('Year', axis=1)
X_train= X_train.drop('gvkey', axis=1)

In [34]:
randforst = RandomForestRegressor(n_estimators=50) 
randforst.fit(X_train, y_train)
#randforst.score(X_test, y_test)
randforst_Pred=randforst.predict(X_test)

In [40]:
F_imp=randforst.feature_importances_
F_imp = pd.DataFrame(randforst.feature_importances_,index = X_train.columns,columns=['imp']).sort_values('imp',ascending=False)
newfeatures=F_imp.iloc[:50,:]
newfeatures=newfeatures.index.tolist()

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mean_squared_error(y_test,randforst_Pred)
mean_absolute_percentage_error(y_test, randforst_Pred)

#Dataset with only new features
X_train_NewF=X_train[newfeatures]
X_test_NewF=X_test[newfeatures]


scaler = StandardScaler()
scaler.fit(X_train_NewF)
StandardScaler(copy=True, with_mean=True, with_std=True)
X_train_NewFt=scaler.transform(X_train_NewF)
X_test_NewFt=scaler.transform(X_test_NewF)

In [ ]:
np.random.seed(10)
classifier = Sequential()
classifier.add(Dense(32, input_dim=50, activation='hard_sigmoid'))
classifier.add(Dense(8, activation='elu'))
classifier.add(Dense(1, activation='relu')) 
classifier.compile(optimizer='adam',loss='mse',  metrics=['accuracy'])
classifier.fit(X_train_NewFt, y_train, batch_size=10,epochs=30)

Epoch 1/30
66774/66774 [==============================] - 8s 126us/step - loss: 0.0041 - acc: 0.0000e+00
Epoch 2/30
66774/66774 [==============================] - 8s 124us/step - loss: 0.0025 - acc: 0.0000e+00
Epoch 3/30
66774/66774 [==============================] - 7s 112us/step - loss: 0.0025 - acc: 0.0000e+00
Epoch 4/30
66774/66774 [==============================] - 8s 122us/step - loss: 0.0025 - acc: 0.0000e+00
Epoch 5/30
66774/66774 [==============================] - 8s 115us/step - loss: 0.0025 - acc: 0.0000e+00
Epoch 6/30
66774/66774 [==============================] - 8s 123us/step - loss: 0.0025 - acc: 0.0000e+00
Epoch 7/30
66774/66774 [==============================] - 8s 126us/step - loss: 0.0025 - acc: 0.0000e+00
Epoch 8/30
66774/66774 [==============================] - 8s 125us/step - loss: 0.0025 - acc: 0.0000e+00
Epoch 9/30
66774/66774 [==============================] - 8s 115us/step - loss: 0.0025 - acc: 0.0000e+00
Epoch 10/30
66774/66774 [==============================

In [1]:
#Calculate mean absolute error
Y_pred = classifier.predict(X_test_NewFt)

mape=mean_absolute_percentage_error(y_test,Y_pred)

mape

NameError: name 'classifier' is not defined